In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow import keras


In [ ]:
data = pd.read_csv('loanapproval.csv')
data.head()

,age,marital,Personal_loan,housing_loan,ever_defaulted,Loan_approved
0,56,married,no,no,no,no
1,57,married,no,no,unknown,no
2,37,married,yes,no,no,no
3,40,married,no,no,no,no
4,56,married,no,yes,no,no


In [ ]:
# Checking for Missing values 
data.isnull().sum()
# convert string variable to One Hot Encoding
dummyfied_data = data.apply(le.fit_transform)
dummyfied_data.head()


,age,marital,Personal_loan,housing_loan,ever_defaulted,Loan_approved
0,39,1,0,0,0,0
1,40,1,0,0,1,0
2,20,1,2,0,0,0
3,23,1,0,0,0,0
4,39,1,0,2,0,0


In [ ]:
# Scalling the numeric column
col_to_scale =['age']
dummyfied_data[col_to_scale] = scaler.fit_transform(dummyfied_data[col_to_scale])
dummyfied_data.head()
X = dummyfied_data.drop('Loan_approved',axis = 1)
Y = dummyfied_data[['Loan_approved']]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, 
random_state = 42)
test_df = pd.concat ([X_test,y_test],axis =1 )
train_df = pd.concat([X_train,y_train],axis = 1)

In [ ]:
# Adaboost classifier
abc = AdaBoostClassifier(random_state=0)
abc_model = abc.fit(X_train, y_train)
abc_pred = abc_model.predict(X_test)
train_pred = abc_model.predict(X_train)
train_df['abc_pred'] = train_pred
test_df['abc_pred'] = abc_pred                  
# test_df.to_csv('classification_results.csv')
print(accuracy_score(y_train, train_pred),accuracy_score(y_test, abc_pred))


C:\Users\Vineel Akash\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8879514415781488 0.8858946346200534


In [ ]:
# creating a RF classifier, training a random Forest model
clf = RandomForestClassifier(random_state = 42) 
clf = clf.fit(X_train, y_train)                 
# Predicting on Test data and checking the accuracy. This can be extended to 
rf_pred = clf.predict(X_test)
train_pred = clf.predict(X_train)
train_df['rf_pred'] = train_pred
test_df['rf_pred'] = rf_pred
print(accuracy_score(y_train, train_pred),accuracy_score(y_test, rf_pred))


C:\Users\Vineel Akash\AppData\Local\Temp\ipykernel_7096\2102841236.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = clf.fit(X_train, y_train)


0.8923520485584219 0.883709638261714


In [ ]:
# Gradient boosting classifier
gbcl = GradientBoostingClassifier(random_state = 42)
gbcl = gbcl.fit(X_train, y_train)
gbcl_pred = gbcl.predict(X_test)
train_pred = gbcl.predict(X_train)
train_df['gbcl_pred'] = train_pred
test_df['gbcl_pred'] = gbcl_pred                   
# test_df.to_csv('classification_results.csv')
print(accuracy_score(y_train, train_pred),accuracy_score(y_test, gbcl_pred))
                   


C:\Users\Vineel Akash\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8894992412746586 0.8852876911871813


In [ ]:
# Xgboost
xgb = XGBClassifier(random_state = 42)
xgb = xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
train_pred = xgb.predict(X_train)
train_df['xgb_pred'] = train_pred
test_df['xgb_pred'] = xgb_pred                   
# test_df.to_csv('classification_results.csv')
print(accuracy_score(y_train, train_pred),accuracy_score(y_test, xgb_pred))

0.8911380880121396 0.8852876911871813


In [ ]:
# Create the network 
model = keras.Sequential([ 
    keras.layers.Dense(3, input_shape = (5,),activation = 'relu'), 
    keras.layers.Dense(2, input_shape = (3,),activation = 'relu'), 
    keras.layers.Dense(1,activation = 'sigmoid'),
])

# Compile the network 
model.compile(optimizer ='adam', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'] 
             )

# train the model 
model.fit(X_train,y_train,epochs = 5)

# Predict based on ANN 
yp_test = model.predict(X_test)
yp_train = model.predict(X_train)

y_pred_train = [] 
for element in yp_train: 
    if element > 0.5: 
        y_pred_train.append(1) 
    else: 
        y_pred_train.append(0)
# Check accuracy of the model 
accuracy_score(y_train, y_pred_train)

y_pred_test = [] 
for element in yp_test: 
    if element > 0.5: 
        y_pred_test.append(1) 
    else: 
        y_pred_test.append(0)
# Check accuracy of the model 
accuracy_score(y_test, y_pred_test)


train_df['ann_pred'] = y_pred_train
test_df['ann_pred'] = y_pred_test 


Epoch 1/5
1030/1030 [==============================] - 5s 3ms/step - loss: 0.4565 - accuracy: 0.8116
Epoch 2/5
1030/1030 [==============================] - 3s 3ms/step - loss: 0.3477 - accuracy: 0.8876
Epoch 3/5
1030/1030 [==============================] - 3s 3ms/step - loss: 0.3460 - accuracy: 0.8876
Epoch 4/5
1030/1030 [==============================] - 3s 3ms/step - loss: 0.3448 - accuracy: 0.8876
Epoch 5/5
258/258 [==============================] - 1s 2ms/step


0.8865015780529255

In [ ]:
# Ensemble the ensembles
column_names = ['abc_pred', 'rf_pred', 'gbcl_pred','xgb_pred','ann_pred']
train_df['sum'] = train_df[column_names].sum(axis=1)
train_df['ensembeled_pred'] = [1 if x > 2 else 0 for x in train_df['sum'] ]
score = accuracy_score(y_train, train_df['ensembeled_pred'])
score

0.8901062215477997

In [ ]:
# Ensemble the ensembles
column_names = ['abc_pred', 'rf_pred', 'gbcl_pred','xgb_pred','ann_pred']
test_df['sum'] = test_df[column_names].sum(axis=1)
test_df['ensembeled_pred'] = [1 if x > 2 else 0 for x in test_df['sum'] ]
score = accuracy_score(y_test, test_df['ensembeled_pred'])
score

0.8855304685603301

In [ ]:
test_df.to_csv("output.csv")